In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%run Mask-Utils.ipynb

In [ ]:
TEXTURE_SIZE = 400

In [ ]:
face_boxes, tddfa = load_models()


In [ ]:
predictor = get_vertices_predictor(face_boxes, tddfa, is_cropped=True)

___

In [ ]:
from skimage.io import imread

# test_image_url = 'https://avatars.mds.yandex.net/get-kinopoisk-post-img/2423210/6fa756128b42aa4553fd3f64339b2748/960x540'
# test_image_url = 'https://i.pinimg.com/736x/ce/1a/d0/ce1ad006c5969585e2bc0609a9981dae.jpg'
test_image_url = 'https://upload.wikimedia.org/wikipedia/commons/6/6c/Lionel_Messi_in_2018.jpg'
# test_image_url = 'https://previews.123rf.com/images/bowie15/bowie151310/bowie15131000036/22756898-screaming-shouting-man.jpg'
# test_image_url = 'https://westcottu.com/wp-content/uploads/robert_close.jpg'
# test_image_url = 'https://cdn.profoto.com/cdn/0521445/contentassets/b61cf60b567f46ac9a274c3f87a5bcb1/portrait_technique_0002.jpg'
# test_image_url = 'https://as1.ftcdn.net/jpg/02/54/74/14/500_F_254741496_XqAP4hpIpL2miUfziPBEXvxsFTMzr3PE.jpg'
# test_image_url = 'https://i.etsystatic.com/15323487/d/il/b394b9/2610037605/il_340x270.2610037605_c2hz.jpg?version=0'
# test_image_url = 'https://i.pinimg.com/originals/b1/d4/b8/b1d4b854b2aa9fe9b6884a2bd8e11125.jpg'
# test_image_url = 'https://www.hopkinsmedicalproducts.com/images/xxl/Medical-Grade-Mask.jpg'
# test_image_url = 'https://images-na.ssl-images-amazon.com/images/I/719krZoENLL._AC_SY679_.jpg'
# test_image_url = 'https://www.meaok.com/media/catalog/product/cache/1c79e4ec97fce480fc1866d4e428246e/p/f/pf07375_pri03.jpg'


test_image = np.ascontiguousarray(imread(test_image_url)[..., ::-1])
imshow(test_image)

In [ ]:
%time ver = predictor(test_image)
ver

In [ ]:
texture = uv_tex(test_image, [ver], tddfa.tri, uv_h=TEXTURE_SIZE, uv_w=TEXTURE_SIZE)
imshow(texture)

In [ ]:
%time uv_coords = get_uv_coords(TEXTURE_SIZE)
uv_coords.shape

In [ ]:
%time uv2ver_sparse = get_sparse_uv2ver(uv_coords, TEXTURE_SIZE, ver)

In [ ]:
cached_linear_nd_creator = CachedInterpolatorCreator(interpolate.LinearNDInterpolator)

In [ ]:
# dense
%time uv2ver = inpaint(uv2ver_sparse, \
                 mask=np.isnan(uv2ver_sparse).astype(np.uint8), \
                 interpolator_cls=cached_linear_nd_creator)

In [ ]:
show_channelwise(uv2ver_sparse)
show_channelwise(uv2ver)

In [ ]:
%time face_indicator = get_face_indicator(uv2ver)
pltshow(face_indicator)

In [ ]:
# %time convex_uv2ver_sparse = get_almost_convex_sparse_uv2ver(ver, uv2ver, face_indicator)

In [ ]:
%time convexer = FixedConvexer(ver, uv_coords, face_indicator, uv2ver)

In [ ]:
%time convex_uv2ver_sparse = convexer(uv2ver)

In [ ]:
cached_linear_nd_ext_creator = CachedInterpolatorCreator(LinearNDInterpolatorExt)

In [ ]:
%time convex_uv2ver = inpaint(convex_uv2ver_sparse, \
                        mask=np.isnan(convex_uv2ver_sparse).astype(np.uint8), \
                        interpolator_cls=cached_linear_nd_ext_creator)

In [ ]:
show_channelwise(convex_uv2ver_sparse)
show_channelwise(convex_uv2ver)

In [ ]:
blurred_convex_uv2ver = blur_multichannel(convex_uv2ver, sigma=(2.5, 5.0), mode='nearest')
show_channelwise(blurred_convex_uv2ver)

In [ ]:
blur_mix_weights = get_blur_weights(convex_uv2ver.shape)[..., None]
show_channelwise(blur_mix_weights)

In [ ]:
convex_uv2ver = convex_uv2ver * (1.0 - blur_mix_weights) + blurred_convex_uv2ver * blur_mix_weights
show_channelwise(convex_uv2ver)

In [ ]:
%time convex_mesh = get_mesh(convex_uv2ver, face_indicator, TEXTURE_SIZE, TEXTURE_SIZE)

___

In [ ]:
%time visible_skin = get_visible_skin(uv2ver)
pltshow(np.where(visible_skin[..., None] == 1, texture / 255.0, np.nan))

In [ ]:
%time target_lightness_ratio = get_cheeks_lightness_ratio(texture, visible_skin)
target_lightness_ratio

In [ ]:
%time light_x = find_light_x_position( \
    target_lightness_ratio, \
    uv_coords, ver, tddfa.tri, texture, visible_skin, \
    render_app, test_image.shape, \
)
light_x

___

In [ ]:
from random import choice

facecover_path = choice(['flat_mask.png',
                         'flat_mask_circular.png',
                         'flat_mask_slipknot.png',
                         'flat_mask_white.png',
                         'flat_mask_green.png',
                         'flat_mask_red.png',
                         'flat_mask_black.png',
                         ])

facecover_texture = cv2.imread(f'masks/{facecover_path}', cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.0
pltshow(facecover_texture)

In [ ]:
%time facecover = FacecoverTextureWarper()(facecover_texture, texture.shape[0])
pltshow(facecover)

In [ ]:
%time facecover_tr = facecover_color_transfer(facecover, texture)
show_channelwise([facecover, facecover_tr])

In [ ]:
facecover_tr[..., :3] = blur_multichannel(facecover_tr[..., :3], sigma=(1.5, 1.5), mode='nearest')
facecover_tr[..., -1:] = blur_multichannel(facecover_tr[..., -1:], sigma=(1.5, 1.5), mode='nearest')

show_channelwise([facecover_tr])

In [ ]:
%time update_colors(convex_mesh, facecover_tr)

In [ ]:
ambient_w, direct_w = get_lightning_params(target_lightness_ratio, light_x, facecover_tr)
ambient_w, direct_w

In [ ]:
%time facecover_render, alpha_render, prefinal_render, final_render = render_facecover( \
    render_app, convex_mesh, test_image, light_x, ambient_w, direct_w, return_intermediate=True \
)

In [ ]:
show_channelwise([facecover_render / 255.0, alpha_render ])

In [ ]:
imshow(prefinal_render)

In [ ]:
imshow(final_render)